# Accessing the Youtube API
This Notebook explores convenience functions for accessing the Youtube API.
Writen by Leon Yin and Megan Brown

In [1]:
import os
import sys
import json
import datetime
import pandas as pd

# this is to import youtube_api from the py directory
sys.path.append(os.path.abspath('../')) 
import youtube_api
from youtube_api import youtube_api_utils as utils

from runtimestamp.runtimestamp import runtimestamp
runtimestamp()

Updated 2018-09-27 17:55:47.834139
By ly501
Using Python 3.6.5
On Linux-3.10.0-514.10.2.el7.x86_64-x86_64-with-centos-7.3.1611-Core


In [2]:
key = os.environ.get('YT_KEY')

We can go from Username to `channel_id`. `channel_id` is required to get uploaded videos, user metadata, and relationships like subscriptions and featured channels.

In [3]:
yt = youtube_api.YoutubeDataApi(key)

In [4]:
yt.get_channel_id_from_user('munchies')

'UCaLfMkkHhSA_LaCta0BzyhQ'

In [5]:
channel_id = 'UCaLfMkkHhSA_LaCta0BzyhQ'

We can collect channel-level metrics and metadata:

In [6]:
channel_meta = yt.get_channel_metadata(channel_id)
channel_meta

OrderedDict([('channel_id', 'UCaLfMkkHhSA_LaCta0BzyhQ'),
             ('title', 'Munchies'),
             ('account_creation_date',
              datetime.datetime(2014, 3, 24, 21, 21, 29)),
             ('keywords',
              'food vice "how to" cooking recipe "fresh off the boat" munchies eating "epic mealtime" tutorial "cooking show"'),
             ('description',
              'MUNCHIES is a website and digital video channel from VICE dedicated to food and its global purpose. Launched in 2014, MUNCHIES offers groundbreaking content from a youth driven perspective. In today\'s modern world, the formerly tangible pleasures of music, film, and emerging media are just one click away. Food and the events that manifest around it are one of the everlasting experiences that cannot be replicated by arcs and zeroes. MUNCHIES chronicles the wide spectrum of the global culinary experience and the diverse voices that are pulling us forward: chefs and home cooks, makers and consumers, the p

Note that `topic_ids` is a json serialized list.

In [7]:
channel_meta['topic_ids']

'https://en.wikipedia.org/wiki/Lifestyle_(sociology)|https://en.wikipedia.org/wiki/Food|https://en.wikipedia.org/wiki/Entertainment'

Note for some API calls that require a "playlist ID", you need to use the playlist id (from uploads or likes) rather than the `channel_id`.

In [8]:
playlist_id = channel_meta['playlist_id_uploads']
playlist_id

'UUaLfMkkHhSA_LaCta0BzyhQ'

In [9]:
channel_id == playlist_id

False

For user uploads the channel ID's first two letters are replaced with "UU" (User Upload) and "LL" (Likes), <br>these relationships are captured in two helper-functions.<br> `yt.get_upload_playlist_id()` and `yt.get_liked_playlist_id()`

In [10]:
utils.get_upload_playlist_id(channel_id)

'UUaLfMkkHhSA_LaCta0BzyhQ'

We can use the following function to get all the video IDs from any playlist id.<br>
`cutoff_date` can be used to filter out videos after a certain date and `stop_after_n_iterations` can be used for testing to return the first N * 50 video ids.

In [11]:
video_ids = yt.get_videos_from_playlist_id(playlist_id, published_after=datetime.datetime(2017,1,1))

In [12]:
df = pd.DataFrame(video_ids)
df.head()

,video_id,channel_id,publish_date,collection_date
0,wmxDZeh8W34,UCaLfMkkHhSA_LaCta0BzyhQ,2018-09-27 15:25:46,2018-09-27 17:55:51.684942
1,N5Cc3fx-JPo,UCaLfMkkHhSA_LaCta0BzyhQ,2018-09-21 13:00:07,2018-09-27 17:55:51.685020
2,PIXQdfiRZNk,UCaLfMkkHhSA_LaCta0BzyhQ,2018-09-25 13:00:08,2018-09-27 17:55:51.685081
3,r2wWEGEMC1M,UCaLfMkkHhSA_LaCta0BzyhQ,2018-09-20 13:00:08,2018-09-27 17:55:51.685140
4,SakUDuMWloI,UCaLfMkkHhSA_LaCta0BzyhQ,2018-09-14 15:30:52,2018-09-27 17:55:51.685198


Let's look at the data we can collect on a video level...

In [13]:
video_id = df['video_id'].tolist()
video_id[:2]

['wmxDZeh8W34', 'N5Cc3fx-JPo']

In [14]:
yt.get_video_metadata(video_id[0])

OrderedDict([('video_id', 'wmxDZeh8W34'),
             ('channel_title', 'Munchies'),
             ('channel_id', 'UCaLfMkkHhSA_LaCta0BzyhQ'),
             ('video_publish_date',
              datetime.datetime(2018, 9, 27, 15, 25, 46)),
             ('video_title', 'How To Dress Up a Frozen Pizza'),
             ('video_description',
              "A frozen pizza is a nearly perfect food, but Cara Nicoletti knows a few tricks to truly take it all the way. Just add sautéed mushrooms, broccoli rabe, caramelized onions, and some fresh ricotta, and suddenly you have the pizza of your dreams. It's not delivery... it's a frozen pizza that you put delicious things on top of. (...We're working on our ad campaign.)\n\nWATCH NEXT: The Pizza Show: Sicily - https://youtu.be/fPHjX6p-RbQ\n\nSubscribe to Munchies here: http://bit.ly/Subscribe-to-MUNCHIES\n\nCheck out http://munchies.tv for more!\n\nFollow Munchies here:\nFacebook: http://facebook.com/munchies\nTwitter: http://twitter.com/munchies\nT

The function also works for a list of up to 50 video ids

In [15]:
video_meta = yt.get_video_metadata(video_id)

In [16]:
len(video_id)

162

In [17]:
df_video_meta = pd.DataFrame(video_meta)
df_video_meta.head(2)

,video_id,channel_title,channel_id,video_publish_date,video_title,video_description,video_category,video_view_count,video_comment_count,video_like_count,video_dislike_count,video_thumbnail,video_tags,collection_date
0,wmxDZeh8W34,Munchies,UCaLfMkkHhSA_LaCta0BzyhQ,2018-09-27 15:25:46,How To Dress Up a Frozen Pizza,"A frozen pizza is a nearly perfect food, but C...",24,16333,224,649,240,https://i.ytimg.com/vi/wmxDZeh8W34/hqdefault.jpg,how to|cooking|Munchies|Munchiestv|food|drinks...,2018-09-27 17:55:54.036701
1,N5Cc3fx-JPo,Munchies,UCaLfMkkHhSA_LaCta0BzyhQ,2018-09-21 13:00:07,How To Make Mustard-Braised Chicken,Full Recipe Here: https://vice.video/2pqy5MJ\n...,24,78323,195,2386,52,https://i.ytimg.com/vi/N5Cc3fx-JPo/hqdefault.jpg,how to|cooking|Munchies|Munchiestv|food|drinks...,2018-09-27 17:55:54.036779


For establishing relationships you can list featured channels and subscriptions

In [18]:
yt.get_featured_channels(channel_id)

[{'UCaLfMkkHhSA_LaCta0BzyhQ': ['UCn8zNIfYAQNdrFRrr8oibKw',
   'UCWF0PiUvUi3Jma2oFgaiX2w',
   'UCfQDD-pbllOCXHYwiXxjJxA',
   'UCZaT_X_mc0BI-djXOlfhqWQ',
   'UCB6PV0cvJpzlcXRG7nz6PpQ',
   'UC0iwHRFpv2_fpojZgQhElEQ',
   'UC_NaA2HkWDT6dliWVcvnkuQ',
   'UCS6R2iiAJ1FvEYl4B3zmljw',
   'UC8C8WuWSsFjWFaTHcUQeQxA',
   'UC9ISPZsMaBi5mutsgX6LC1g',
   'UCiZCX1R1F3xYGbeXq1JscKA',
   'UCVfmHpXONv-LVACBV68tq5Q',
   'UC5e0xSqwDGlRg3sdvGQh7lg',
   'UClW2OsdCa2E_KkLZNpm_9VQ',
   'UC9XpoCBNvStSmp3gVf_jG1g',
   'UCflb1gG-X1dy1Ru5JIk5sPw',
   'UCNDUud96oGK5xQ9gyg913vw']}]

You can save on time by using a list of inputs for some api calls:

In [19]:
channel_ids = ['UCaLfMkkHhSA_LaCta0BzyhQ', 'UC6MFZAOHXlKK1FI7V0XQVeA']

In [21]:
yt.get_featured_channels(channel_ids)

[{'UCaLfMkkHhSA_LaCta0BzyhQ': ['UCn8zNIfYAQNdrFRrr8oibKw',
   'UCWF0PiUvUi3Jma2oFgaiX2w',
   'UCfQDD-pbllOCXHYwiXxjJxA',
   'UCZaT_X_mc0BI-djXOlfhqWQ',
   'UCB6PV0cvJpzlcXRG7nz6PpQ',
   'UC0iwHRFpv2_fpojZgQhElEQ',
   'UC_NaA2HkWDT6dliWVcvnkuQ',
   'UCS6R2iiAJ1FvEYl4B3zmljw',
   'UC8C8WuWSsFjWFaTHcUQeQxA',
   'UC9ISPZsMaBi5mutsgX6LC1g',
   'UCiZCX1R1F3xYGbeXq1JscKA',
   'UCVfmHpXONv-LVACBV68tq5Q',
   'UC5e0xSqwDGlRg3sdvGQh7lg',
   'UClW2OsdCa2E_KkLZNpm_9VQ',
   'UC9XpoCBNvStSmp3gVf_jG1g',
   'UCflb1gG-X1dy1Ru5JIk5sPw',
   'UCNDUud96oGK5xQ9gyg913vw']},
 {'UC6MFZAOHXlKK1FI7V0XQVeA': ['UCSHsNH4FZXFeSQMJ56AdrBA']}]

Subscriptions can only done one channel at a time:

In [22]:
yt.get_subscriptions(channel_id)

[OrderedDict([('subscription_title', 'VICE en Español .'),
              ('subscription_channel_id', 'UCYB6UxeSTHJyEMq-clobnrg'),
              ('subscription_kind', 'youtube#channel'),
              ('subscription_publish_date',
               datetime.datetime(2016, 10, 4, 17, 43, 39)),
              ('collection_date',
               datetime.datetime(2018, 9, 27, 17, 56, 14, 814517))]),
 OrderedDict([('subscription_title', 'YouTube Nation'),
              ('subscription_channel_id', 'UCUD4yDVyM54QpfqGJX4S7ng'),
              ('subscription_kind', 'youtube#channel'),
              ('subscription_publish_date',
               datetime.datetime(2014, 8, 28, 19, 14, 49)),
              ('collection_date',
               datetime.datetime(2018, 9, 27, 17, 56, 14, 814575))]),
 OrderedDict([('subscription_title', 'VICE .'),
              ('subscription_channel_id', 'UCn8zNIfYAQNdrFRrr8oibKw'),
              ('subscription_kind', 'youtube#channel'),
              ('subscription_publish_dat

Subscriptions can be more descriptive by setting the `descriptive` flag as True.

In [23]:
yt.get_subscriptions(channel_id)[:2]

[OrderedDict([('subscription_title', 'VICE en Español .'),
              ('subscription_channel_id', 'UCYB6UxeSTHJyEMq-clobnrg'),
              ('subscription_kind', 'youtube#channel'),
              ('subscription_publish_date',
               datetime.datetime(2016, 10, 4, 17, 43, 39)),
              ('collection_date',
               datetime.datetime(2018, 9, 27, 17, 56, 28, 531035))]),
 OrderedDict([('subscription_title', 'YouTube Nation'),
              ('subscription_channel_id', 'UCUD4yDVyM54QpfqGJX4S7ng'),
              ('subscription_kind', 'youtube#channel'),
              ('subscription_publish_date',
               datetime.datetime(2014, 8, 28, 19, 14, 49)),
              ('collection_date',
               datetime.datetime(2018, 9, 27, 17, 56, 28, 531094))])]

You can also get the comments for a given video

In [24]:
yt.get_video_comments(video_id[0], max_results=2)

[OrderedDict([('video_id', 'wmxDZeh8W34'),
              ('commenter_channel_url',
               'http://www.youtube.com/channel/UCgUewvD0u0Kr9pr1JpiXmkw'),
              ('commenter_channel_display_name', 'Noma'),
              ('comment_id', 'Ugx3elfeZbnFxqUrZKF4AaABAg'),
              ('comment_like_count', 0),
              ('comment_publish_date',
               datetime.datetime(2018, 9, 27, 21, 32, 28)),
              ('text', 'hey Cara, why does your necklace say Gabriela?'),
              ('commenter_rating', 'none'),
              ('comment_parent_id', None),
              ('collection_date',
               datetime.datetime(2018, 9, 27, 17, 56, 39, 470124)),
              ('reply_count', 0)]),
 OrderedDict([('video_id', 'wmxDZeh8W34'),
              ('commenter_channel_url',
               'http://www.youtube.com/channel/UCv2Wir-dLGSQ2zLl9rL5uBw'),
              ('commenter_channel_display_name', 'teamSHiTpiG'),
              ('comment_id', 'UgwHQ8pDsMObB7zVNgZ4AaABAg'),
  

For more text we can get closed captions!

In [43]:
video_id = 'wmxDZeh8W34'

In [42]:
captions = yt.get_captions(video_id)

In [44]:
captions

{'caption': "I have put some despicable things onto frozen pizza like some really fancy things that I shouldn't even admit I have put a black truffle on a frozen pizza but I did not pay for that black truffle and I was don't [Music] I'm car Nicoletti today I am in the munchies Test Kitchen and I'm gonna show you how to doctor up a frozen pizza you can trick people into thinking you're an amazing cook for this rendition today I'm going to use a variety of wild mushrooms some broccoli rabe I'm gonna caramelize some onions and then I have some fresh ricotta that I'm gonna dollop on usually frozen pizzas will come with round of cardboard underneath and I'm doing it on a sheet pan I think sometimes the cardboard makes it taste like cardboard so I'm gonna put this in the oven right now for about 25 minutes and while it's cooking I'm gonna cook all the toppings just to make this even easier I'm gonna try to do everything in one pan I'm not gonna try I'm gonna do everything in one pan first be

You can also get the recommended videos for any given video

In [45]:
recommended_vids = yt.get_recommended_videos(video_id)

In [46]:
recommended_vids[:2]

[OrderedDict([('video_id', 'SakUDuMWloI'),
              ('channel_title', 'Munchies'),
              ('channel_id', 'UCaLfMkkHhSA_LaCta0BzyhQ'),
              ('video_publish_date',
               datetime.datetime(2018, 9, 13, 23, 2, 35)),
              ('video_title',
               'How To Make Hennessy Shrimp 3 Ways with Meyhem Lauren'),
              ('video_description',
               'Meyhem Lauren is back in the MUNCHIES test kitchen to make his dreams come true by combining his two eternal loves: Hennessy and shrimp. \n\nPrepare to have your mind blown with not just one, but three, Hennessy and shrimp recipes: Hennessy-glazed shrimp, Hennessy shrimp au poivre, and crab stuffed shrimp.\n\nLinks to all the recipes below: \n\nEasy Glazed Shrimp: https://vice.video/2NdFEVt\n\nCrab Stuffed Shrimp: https://vice.video/2OlQSDU\n\nShrimp and Hennessy Pasta: https://vice.video/2OmjahD\n\nSubscribe to Munchies here: http://bit.ly/Subscribe-to-MUNCHIES\n\nCheck out http://munchies.tv fo